<a href="https://colab.research.google.com/github/frapasti/Embedded-Person-detection/blob/main/Scale_Stat_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scale statistics evaluation
This notebook aims to evaluate the performances of some off the shelf person detectors with metrics such as precision, recall, miss rate and f1 score over different input scales.

First of all, install the needed libraries

In [ ]:
!pip install fiftyone --no-binary fiftyone,voxel51-eta
import fiftyone as fo
import fiftyone.zoo as foz
import numpy as np
from numpy import expand_dims
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import json
import cv2
import imutils
from google.colab.patches import cv2_imshow
from keras.preprocessing.image import load_img
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import os

def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

# DATASET DOWNLOAD
In order to evaluate over different scales the input images must start from the same size.
The chosen size is 640x480.
The dataset is downloaded with the fiftyone tool for coco-2017

In [3]:
#DATASET containing persons
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    classes=["person"],
    only_matching = True,
    max_samples=1000,
)

 100% |██████|    1.9Gb/1.9Gb [2.8s elapsed, 0s remaining, 695.0Mb/s]       
Extracting annotations to '/root/fiftyone/coco-2017/raw/instances_val2017.json'
 100% |████████████████| 1000/1000 [1.5m elapsed, 0s remaining, 11.5 images/s]      
Writing annotations for 1000 downloaded samples to '/root/fiftyone/coco-2017/validation/labels.json'
Dataset info written to '/root/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'validation'
 100% |███████████████| 1000/1000 [5.0s elapsed, 0s remaining, 206.4 samples/s]      
Dataset 'coco-2017-validation-1000' created


In [5]:
f = open('/root/fiftyone/coco-2017/validation/labels.json')
data = json.load(f)
annotations = data['annotations']
every_path = []
cols = 10
rows = 1000
bbox_size = 4
counters = [0] * rows
bboxes = [[] for i in range(rows)] 
items = 0
# Creation of the list of paths of images in the dataset and bboxes
# This elements are extracted from the json file downloaded from coco with the dataset
for entry in data['annotations']:
    class_id = entry['category_id']
    if (class_id == 1):
        img_id = entry['image_id']
        path = float(img_id)/pow(10,11)
        dec_path = format(path, '.11f')
        final_path = ''
        final_path = '/root/fiftyone/coco-2017/validation/data/'+ dec_path.replace('.', '') + '.jpg'
        if(every_path.count(final_path) == 0):
          every_path.append(final_path)
          bboxes[items].append(entry['bbox'])
          items = items + 1
        else:
          index = every_path.index(final_path)
          if(counters[index] + 1 < 10):
            bboxes[index].append(entry['bbox'])
            counters[index] = counters[index] + 1

# Matain only images of size 640x480
f_every_path = []
f_bboxes = []
for i in range(0, len(every_path)):
  original_img = load_img(every_path[i])
  width, height = original_img.size
  if(width == 640 and height == 480):
    f_every_path.append(every_path[i])
    f_bboxes.append(bboxes[i])

assert(len(bboxes)==len(every_path))

In [ ]:
#DATASET of non persons
dataset1 = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=["detections"],
    only_matching = True,
    max_samples=500,
)

class_name = "Person"

# Find samples that have a "Person"
pos_view = dataset1.filter_labels("ground_truth", F("label")==class_name)

# Find all samples without a positively labeled "Person"
neg_view = dataset1.exclude(pos_view)

# Tag any samples that have a person in the App with "remove"
session = fo.launch_app(view=neg_view)

Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'
  16% |██/---------------|  80/500 [6.8s elapsed, 35.6s remaining, 11.7 images/s]    

In [7]:
# Find and remove all tagged samples from the DatasetView
neg_view = neg_view.match_tags("remove", bool=False)

neg_view.export(
    export_dir="/content/nonPerson1",
    dataset_type=fo.types.COCODetectionDataset,
)

# Mantain only images of size 640x480
non_persons = listdir_fullpath("/content/nonPerson1/data")
for i in range(0, len(non_persons)):
  original_img = load_img(non_persons[i])
  width, height = original_img.size
  if(width == 640 and height == 480):
    f_every_path.append(non_persons[i])
    f_bboxes.append([])

 100% |█████████████████| 461/461 [1.8s elapsed, 0s remaining, 263.4 samples/s]         


In [10]:
print(len(f_every_path))
print(len(f_bboxes))

224
224
